In [1]:
# Convert CSV to JSON and load JSON into SQL Database
# Dependencies
import pandas as pd
import json
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy import Column, Integer, String, Float, Date
import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy import create_engine

In [2]:
# read weather csv file into dataframe
sale_weather_file = "../Resources/Weather_Sales_Data.csv"
sale_weather_df = pd.read_csv(sale_weather_file)
sale_weather_df.head()

,date,TRANSACTION_TYPE,TOTAL_TRANSACTIONS,TOTAL_SALES,TOTAL_UNITS,TOTAL_COUPONS,max_temp,min_temp,avg_temp,precipitation,snow_fall,snow_depth
0,2016-01-01,S,1604,101482.59,8046,769,42,34,38.0,0.0,0.0,0.0
1,2016-01-02,S,2669,174602.55,13305,1446,40,32,36.0,0.0,0.0,0.0
2,2016-01-03,S,2298,152357.03,11795,1324,45,35,40.0,0.0,0.0,0.0
3,2016-01-04,S,1967,109492.91,9075,947,36,14,25.0,0.0,0.0,0.0
4,2016-01-05,S,1796,108041.47,8424,662,29,11,20.0,0.0,0.0,0.0


In [3]:
class Weather(Base):
    __tablename__ = 'weatherJSON'
    date = Column(String(24), primary_key=True)
    total_transactions = Column(Integer)
    total_sales = Column(Float)
    total_units = Column(Integer)
    total_coupons = Column(Integer)
    total_return_sales = Column(Float)
    total_return_units = Column(Integer)
    max_temp = Column(Integer)
    min_temp = Column(Float)
    avg_temp = Column(Float)
    precipitation = Column(Float)
    snow_fall = Column(Float)
    snow_depth = Column(Float)

In [4]:
# create database connection (need to enter mysql password)
connection_string = "root:raynor1128@localhost/weather_data"
engine = create_engine(f'mysql://{connection_string}')

In [5]:
# confirm tables
engine.table_names()

['sales_2016', 'weather_2016', 'weather_sales_2016']

In [6]:
Base.metadata.create_all(engine)

In [7]:
weatherJSON = sale_weather_df.to_json(orient='index', date_format = 'iso')
weatherJSON = json.loads(weatherJSON)
inds = sale_weather_df.index.values


In [8]:
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [9]:
for ind in inds:
    ind = str(ind)
    dateD = weatherJSON[ind]['date']
    total_transactionsD = weatherJSON[ind]['TOTAL_TRANSACTIONS']
    total_salesD = weatherJSON[ind]['TOTAL_SALES']
    total_unitsD = weatherJSON[ind]['TOTAL_UNITS']
    total_couponsD = weatherJSON[ind]['TOTAL_COUPONS']
    max_tempD = weatherJSON[ind]['max_temp']
    min_tempD = weatherJSON[ind]['min_temp']
    avg_tempD = weatherJSON[ind]['avg_temp']
    precipitationD =weatherJSON[ind]['precipitation']
    snow_fallD = weatherJSON[ind]['snow_fall']
    snow_depthD = weatherJSON[ind]['snow_depth']
    record = Weather(date = dateD, total_transactions = total_transactionsD, total_sales = total_salesD, total_units = total_unitsD, total_coupons = total_couponsD, max_temp = max_tempD, min_temp = min_tempD, avg_temp = avg_tempD, precipitation = precipitationD, snow_fall = snow_fallD, snow_depth = snow_depthD)
    session.add(record)
    session.commit()